<a href="https://colab.research.google.com/github/korakoe/StyleTTS2/blob/main/Colab/StyleTTS2_Finetune_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install packages and download models

In [1]:
# @title Mount Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
!pip install soundfile==0.10.3.post1

  Attempting uninstall: soundfile
    Found existing installation: soundfile 0.12.1
    Uninstalling soundfile-0.12.1:
      Successfully uninstalled soundfile-0.12.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
librosa 0.10.1 requires soundfile>=0.12.1, but you have soundfile 0.10.3.post1 which is incompatible.


In [2]:
%%shell
git clone https://github.com/yl4579/StyleTTS2.git
cd StyleTTS2
pip install soundfile==0.10.3.post1 torchaudio munch torch pydub pyyaml librosa nltk matplotlib accelerate transformers phonemizer einops einops-exts tqdm typing-extensions git+https://github.com/resemble-ai/monotonic_align.git
sudo apt-get install espeak-ng
git-lfs clone https://huggingface.co/yl4579/StyleTTS2-LibriTTS
mv StyleTTS2-LibriTTS/Models .

Cloning into 'StyleTTS2'...
remote: Enumerating objects: 308, done.
remote: Counting objects: 100% (147/147), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 308 (delta 125), reused 86 (delta 82), pack-reused 161
Receiving objects: 100% (308/308), 133.94 MiB | 33.32 MiB/s, done.
Resolving deltas: 100% (167/167), done.
Updating files: 100% (46/46), done.
  Cloning https://github.com/resemble-ai/monotonic_align.git to /tmp/pip-req-build-0uljowas
  Running command git clone --filter=blob:none --quiet https://github.com/resemble-ai/monotonic_align.git /tmp/pip-req-build-0uljowas
  Resolved https://github.com/resemble-ai/monotonic_align.git to commit 78b985be210a03d08bc3acc01c4df0442105366f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 6.0 MB/s

### Download dataset (LJSpeech, 200 samples, ~15 minutes of data)

You can definitely do it with fewer samples. This is just a proof of concept with 200 smaples.

In [3]:
%cd StyleTTS2
!rm -rf Data

/content/StyleTTS2


In [ ]:
!gdown --id 1vqz26D3yn7OXS2vbfYxfSnpLS6m6tOFP
!unzip Data.zip

### Convert Unphonemized (only do if you're using a text based ljspeech dataset)

In [9]:
# @title Convert
path_to_wavs_txt = "/content/drive/Othercomputers/My Laptop/Voice Clones/Me/wavs.txt" # @param {type:"string"}
save_to_path = "/content/drive/Othercomputers/My Laptop/Voice Clones/Me/Train_list.txt" # @param {type:"string"}
import phonemizer
from tqdm import tqdm

with open(path_to_wavs_txt, 'r', encoding="utf8") as fh:
    lines = fh.readlines()

final = []
for line in tqdm(lines):
    data = line.replace("\n", "").split("|")
    data[1] = phonemizer.phonemize(data[1], backend="espeak", with_stress=True, preserve_punctuation=True, njobs=4, strip=True)
    if len(data) < 3:
        data.append(0)

    final.append(f"{data[0]}|{data[1]}|{data[2]}")

with open(save_to_path, 'w', encoding="utf8") as fh:
    final = "\n".join(final)
    fh.write(final)

100%|██████████| 140/140 [00:12<00:00, 11.18it/s]



### Change the finetuning config

Depending on the GPU you got, you may want to change the bacth size, max audio length, epiochs and so on.

In [10]:
config_path = "Configs/config_ft.yml"

import yaml
config = yaml.safe_load(open(config_path))

In [15]:
# @title Training Config
dataset_path = "/content/drive/Othercomputers/My Laptop/Voice Clones/Me/" # @param {type:"string"}
train_list_path = "/content/drive/Othercomputers/My Laptop/Voice Clones/Me/Train_list.txt" # @param {type:"string"}
val_list_path = "/content/drive/Othercomputers/My Laptop/Voice Clones/Me/Val_list.txt" # @param {type:"string"}
OOD_text_path = "/content/drive/Othercomputers/My Laptop/Voice Clones/Me/OOD_texts.txt" # @param {type:"string"}
log_dir = "/content/drive/Othercomputers/My Laptop/Voice Clones/Me/" # @param {type:"string"}

batch_size = 2 # @param {type:"integer"}
save_every = 5 # @param {type:"integer"}
max_len = 100 # @param {type:"integer"}
epochs = 100 # @param {type:"integer"}
learning_rate = 2 # @param {type:""}
joint_epoch = 0.0001 # @param {type:"number"}
config['data_params']['root_path'] = dataset_path
config['data_params']['log_dir'] = log_dir
config['data_params']['train_data'] = train_list_path
config['data_params']['val_data'] = val_list_path
config['data_params']['OOD_data'] = OOD_text_path

config['batch_size'] = batch_size # not enough RAM
config['max_len'] = max_len # not enough RAM
config['epochs'] = epochs
config['save_freq'] = save_every
config['loss_params']['joint_epoch'] = joint_epoch # we do not do SLM adversarial training due to not enough RAM
config['optimizer_params']['lr'] = learning_rate

with open(config_path, 'w') as outfile:
  yaml.dump(config, outfile, default_flow_style=True)

### Start finetuning


In [16]:
!python train_finetune.py --config_path ./Configs/config_ft.yml

2023-11-26 07:09:23.557205: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-26 07:09:23.557257: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-26 07:09:23.557292: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-26 07:09:23.564598: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-26 07:09:24.660311: W tensorflow/c

### Test the model quality

Note that this mainly serves as a proof of concept due to RAM limitation of free Colab instances. A lot of settings are suboptimal. In the future when DDP works for train_second.py, we will also add mixed precision finetuning to save time and RAM. You can also add SLM adversarial training run if you have paid Colab services (such as A100 with 40G of RAM).

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
import torch
torch.manual_seed(0)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

import random
random.seed(0)

import numpy as np
np.random.seed(0)

# load packages
import time
import random
import yaml
from munch import Munch
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
import torchaudio
import librosa
from nltk.tokenize import word_tokenize

from models import *
from utils import *
from text_utils import TextCleaner
textclenaer = TextCleaner()

%matplotlib inline

to_mel = torchaudio.transforms.MelSpectrogram(
    n_mels=80, n_fft=2048, win_length=1200, hop_length=300)
mean, std = -4, 4

def length_to_mask(lengths):
    mask = torch.arange(lengths.max()).unsqueeze(0).expand(lengths.shape[0], -1).type_as(lengths)
    mask = torch.gt(mask+1, lengths.unsqueeze(1))
    return mask

def preprocess(wave):
    wave_tensor = torch.from_numpy(wave).float()
    mel_tensor = to_mel(wave_tensor)
    mel_tensor = (torch.log(1e-5 + mel_tensor.unsqueeze(0)) - mean) / std
    return mel_tensor

def compute_style(path):
    wave, sr = librosa.load(path, sr=24000)
    audio, index = librosa.effects.trim(wave, top_db=30)
    if sr != 24000:
        audio = librosa.resample(audio, sr, 24000)
    mel_tensor = preprocess(audio).to(device)

    with torch.no_grad():
        ref_s = model.style_encoder(mel_tensor.unsqueeze(1))
        ref_p = model.predictor_encoder(mel_tensor.unsqueeze(1))

    return torch.cat([ref_s, ref_p], dim=1)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# load phonemizer
import phonemizer
global_phonemizer = phonemizer.backend.EspeakBackend(language='en-us', preserve_punctuation=True,  with_stress=True)

config = yaml.safe_load(open("Models/LJSpeech/config_ft.yml"))

# load pretrained ASR model
ASR_config = config.get('ASR_config', False)
ASR_path = config.get('ASR_path', False)
text_aligner = load_ASR_models(ASR_path, ASR_config)

# load pretrained F0 model
F0_path = config.get('F0_path', False)
pitch_extractor = load_F0_models(F0_path)

# load BERT model
from Utils.PLBERT.util import load_plbert
BERT_path = config.get('PLBERT_dir', False)
plbert = load_plbert(BERT_path)

model_params = recursive_munch(config['model_params'])
model = build_model(model_params, text_aligner, pitch_extractor, plbert)
_ = [model[key].eval() for key in model]
_ = [model[key].to(device) for key in model]

In [ ]:
files = [f for f in os.listdir("Models/LJSpeech/") if f.endswith('.pth')]
sorted_files = sorted(files, key=lambda x: int(x.split('_')[-1].split('.')[0]))

In [ ]:
params_whole = torch.load("Models/LJSpeech/" + sorted_files[-1], map_location='cpu')
params = params_whole['net']

In [ ]:
for key in model:
    if key in params:
        print('%s loaded' % key)
        try:
            model[key].load_state_dict(params[key])
        except:
            from collections import OrderedDict
            state_dict = params[key]
            new_state_dict = OrderedDict()
            for k, v in state_dict.items():
                name = k[7:] # remove `module.`
                new_state_dict[name] = v
            # load params
            model[key].load_state_dict(new_state_dict, strict=False)
#             except:
#                 _load(params[key], model[key])
_ = [model[key].eval() for key in model]

In [ ]:
from Modules.diffusion.sampler import DiffusionSampler, ADPM2Sampler, KarrasSchedule

In [ ]:
sampler = DiffusionSampler(
    model.diffusion.diffusion,
    sampler=ADPM2Sampler(),
    sigma_schedule=KarrasSchedule(sigma_min=0.0001, sigma_max=3.0, rho=9.0), # empirical parameters
    clamp=False
)

In [ ]:
def inference(text, ref_s, alpha = 0.3, beta = 0.7, diffusion_steps=5, embedding_scale=1):
    text = text.strip()
    ps = global_phonemizer.phonemize([text])
    ps = word_tokenize(ps[0])
    ps = ' '.join(ps)
    tokens = textclenaer(ps)
    tokens.insert(0, 0)
    tokens = torch.LongTensor(tokens).to(device).unsqueeze(0)

    with torch.no_grad():
        input_lengths = torch.LongTensor([tokens.shape[-1]]).to(device)
        text_mask = length_to_mask(input_lengths).to(device)

        t_en = model.text_encoder(tokens, input_lengths, text_mask)
        bert_dur = model.bert(tokens, attention_mask=(~text_mask).int())
        d_en = model.bert_encoder(bert_dur).transpose(-1, -2)

        s_pred = sampler(noise = torch.randn((1, 256)).unsqueeze(1).to(device),
                                          embedding=bert_dur,
                                          embedding_scale=embedding_scale,
                                            features=ref_s, # reference from the same speaker as the embedding
                                             num_steps=diffusion_steps).squeeze(1)


        s = s_pred[:, 128:]
        ref = s_pred[:, :128]

        ref = alpha * ref + (1 - alpha)  * ref_s[:, :128]
        s = beta * s + (1 - beta)  * ref_s[:, 128:]

        d = model.predictor.text_encoder(d_en,
                                         s, input_lengths, text_mask)

        x, _ = model.predictor.lstm(d)
        duration = model.predictor.duration_proj(x)

        duration = torch.sigmoid(duration).sum(axis=-1)
        pred_dur = torch.round(duration.squeeze()).clamp(min=1)

        pred_aln_trg = torch.zeros(input_lengths, int(pred_dur.sum().data))
        c_frame = 0
        for i in range(pred_aln_trg.size(0)):
            pred_aln_trg[i, c_frame:c_frame + int(pred_dur[i].data)] = 1
            c_frame += int(pred_dur[i].data)

        # encode prosody
        en = (d.transpose(-1, -2) @ pred_aln_trg.unsqueeze(0).to(device))
        if model_params.decoder.type == "hifigan":
            asr_new = torch.zeros_like(en)
            asr_new[:, :, 0] = en[:, :, 0]
            asr_new[:, :, 1:] = en[:, :, 0:-1]
            en = asr_new

        F0_pred, N_pred = model.predictor.F0Ntrain(en, s)

        asr = (t_en @ pred_aln_trg.unsqueeze(0).to(device))
        if model_params.decoder.type == "hifigan":
            asr_new = torch.zeros_like(asr)
            asr_new[:, :, 0] = asr[:, :, 0]
            asr_new[:, :, 1:] = asr[:, :, 0:-1]
            asr = asr_new

        out = model.decoder(asr,
                                F0_pred, N_pred, ref.squeeze().unsqueeze(0))


    return out.squeeze().cpu().numpy()[..., :-50] # weird pulse at the end of the model, need to be fixed later

### Synthesize speech

In [ ]:
text = '''Maltby and Company would issue warrants on them deliverable to the importer, and the goods were then passed to be stored in neighboring warehouses.
'''

In [ ]:
# get a random reference in the training set, note that it doesn't matter which one you use
path = "Data/wavs/LJ001-0110.wav"
# this style vector ref_s can be saved as a parameter together with the model weights
ref_s = compute_style(path)

In [ ]:
start = time.time()
wav = inference(text, ref_s, alpha=0.9, beta=0.9, diffusion_steps=10, embedding_scale=1)
rtf = (time.time() - start) / (len(wav) / 24000)
print(f"RTF = {rtf:5f}")
import IPython.display as ipd
display(ipd.Audio(wav, rate=24000, normalize=False))